In [81]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import chart_studio
import chart_studio.plotly as py

In [82]:
pd.set_option("display.max_columns", 500)

In [83]:
file = open("../../plotly_api_key.txt")

api_key = file.read().replace("\n", " ")
file.close()

In [84]:
username = 'matttparker'

In [85]:
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [144]:
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
        SELECT COUNT(*)
        FROM referrals
        """ 

    test = pd.read_sql(query, db)

display(test)

,COUNT(*)
0,6436


In [149]:
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
        SELECT *
        FROM referrals
        LIMIT 1
        """ 

    test = pd.read_sql(query, db)

display(test)

,Unnamed: 0,index,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,from_zip,from_npi_specialty,from_entity_type_code,to_zip,to_npi_specialty,to_entity_type_code,to_facility,to_facility_group,to_facility_name_normalised
0,0,615039,1013179860,1417938846,71,82,35.049,42.548,37075,Internal Medicine,1.0,37083,General Acute Care Hospital,2.0,"MACON COUNTY GENERAL HOSPITAL, INC.",Macon County General Hospital,Macon County General Hospital


In [150]:
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
        SELECT SUM(patient_count), SUM(transaction_count), 
        COUNT(DISTINCT to_npi), COUNT(DISTINCT from_npi)
        FROM referrals
        WHERE from_npi_specialty NOT IN ('Radiology',
                             'Pathology',
                             'Anesthesiology',
                             'Obstetrics & Gynecology',
                             'Pediatrics',
                             'Clinical Pharmacology',
                             'Anesthesiologist Assistant')
        """ 

    test = pd.read_sql(query, db)

display(test)

,SUM(patient_count),SUM(transaction_count),COUNT(DISTINCT to_npi),COUNT(DISTINCT from_npi)
0,949373,1413178,25,3618


In [155]:
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
        SELECT provider_last_name, referrals.*, Specialization
        FROM referrals
        JOIN npidata_nashville
        ON referrals.from_npi = npidata_nashville.npi
        WHERE provider_last_name = 'RIDDICK'
        AND referrals.to_facility_group <> 'Vanderbilt University Medical Center'
        --GROUP BY from_npi, provider_last_name
        """ 

    test = pd.read_sql(query, db)

display(test)

,provider_last_name,Unnamed: 0,index,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,from_zip,from_npi_specialty,from_entity_type_code,to_zip,to_npi_specialty,to_entity_type_code,to_facility,to_facility_group,to_facility_name_normalised,specialization
0,RIDDICK,685,11213125,1417131715,1265487193,90,108,40.833,56.725,37203,Internal Medicine,1.0,37015,General Acute Care Hospital,2.0,"HCA HEALTH SERVICES OF TENNESSEE, INC.",HCA,Ashland City Medical Center HCA,Interventional Cardiology
1,RIDDICK,2115,7107136,1417131715,1023055126,4685,6245,2.890,12.170,37203,Internal Medicine,1.0,37203,General Acute Care Hospital,2.0,"HCA HEALTH SERVICES OF TENNESSEE, INC.",HCA,Centennial Medical Center HCA,Interventional Cardiology
2,RIDDICK,2532,12179199,1417131715,1932146032,64,67,14.075,29.592,37203,Internal Medicine,1.0,37203,General Acute Care Hospital,2.0,"HCA HEALTH SERVICES OF TENNESSEE, INC.",HCA,Centennial Medical Center HCA,Interventional Cardiology
3,RIDDICK,2602,12089923,1417131715,1871530832,174,199,47.327,57.092,37203,Internal Medicine,1.0,37055,General Acute Care Hospital,2.0,CENTRAL TENNESSEE HOSPITAL CORPORATION,HCA,TriStar Horizon Medical Center HCA,Interventional Cardiology
4,RIDDICK,2989,11210309,1417131715,1265445506,182,204,47.417,62.248,37203,Internal Medicine,1.0,37067,General Acute Care Hospital,2.0,WILLIAMSON COUNTY HOSPITAL DISTRICT,Williamson Medical Center,Williamson County Hospital,Interventional Cardiology


## Create Visual of NPIs to Poach - NOT IN PRESENTATION

In [86]:
# From below, summarize by Facility Group
# Find hospital groups with the largest number of single group, non-Vanderbilt NPIs
# Remove Radiology
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
        WITH CTE AS (
            SELECT from_npi
                , COUNT(DISTINCT to_facility_group) AS to_groups
                , SUM(transaction_count) AS npi_transactions
                , SUM(patient_count) AS npi_patients
                , to_facility_group
            FROM referrals
            WHERE to_facility_group <> 'Vanderbilt University Medical Center'
            AND from_npi_specialty NOT IN ('Radiology', 'Emergency Medicine', 'Orthopaedic Surgery',
                                            'Pathology', 'Anesthesiology')
            GROUP BY from_npi
            HAVING to_groups = 1 AND npi_transactions >= 500
        ), CTE_2 AS(
            SELECT CTE.to_facility_group, CTE.npi_transactions, CTE.npi_patients, n.*
            FROM CTE
            INNER JOIN npidata_nashville AS n 
                ON CTE.from_npi = n.npi
            ORDER BY CTE.npi_transactions DESC
        )
        SELECT to_facility_group AS Facility_Group
            , npi
            , provider_first_name || ' ' || provider_last_name AS Name
            , classification AS Classification
            , COALESCE(specialization, 'None') AS Specialization
            , npi_transactions AS NPI_Transactions
            , npi_patients AS NPI_Patients
            , provider_business_address_1
            , provider_business_address_2
            , provider_business_city
            , provider_business_state
            , provider_business_zip5
        FROM CTE_2
        ORDER BY to_facility_group
        """ 

    poach = pd.read_sql(query, db)

display(poach)

,Facility_Group,npi,Name,Classification,Specialization,NPI_Transactions,NPI_Patients,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip5
0,Ascension Saint Thomas,1770519704,DAVID GIBSON,Internal Medicine,Cardiovascular Disease,1648,1454,4230 HARDING RD,SUITE 330,NASHVILLE,TN,37205
1,Ascension Saint Thomas,1720241185,WARREN STRIBLING,Internal Medicine,Advanced Heart Failure and Transplant Cardiology,1648,711,4230 HARDING RD.,STE. 330,NASHVILLE,TN,37205
2,Ascension Saint Thomas,1598719536,DON CHOMSKY,Internal Medicine,Cardiovascular Disease,1621,701,4230 HARDING RD,SUITE 330,NASHVILLE,TN,37205
3,Ascension Saint Thomas,1457317257,GUY MIOTON,Internal Medicine,Cardiovascular Disease,1458,1155,1840 MEDICAL CENTER PKWY,SUITE 201,MURFREESBORO,TN,37129
4,Ascension Saint Thomas,1942429816,ANDREW ZURICK,Internal Medicine,Cardiovascular Disease,1421,1265,4230 HARDING RD.,SUITE 330,NASHVILLE,TN,37205
...,...,...,...,...,...,...,...,...,...,...,...,...
279,Williamson Medical Center,1346397130,PAUL FLESER,Surgery,Vascular Surgery,570,330,100 COVEY DR,SUITE 204,FRANKLIN,TN,37067
280,Williamson Medical Center,1952771941,JAYANTHI SAMUEL,Nurse Practitioner,Family,519,467,4323 CAROTHERS PKWY,SUITE 205,FRANKLIN,TN,37067
281,Williamson Medical Center,1033246640,OUIDA COLLINS,Family Medicine,None,510,407,3601 TVC,None,NASHVILLE,TN,37232
282,Williamson Medical Center,1063864775,PAULA DUNN,Family Medicine,None,504,237,4091 MALLORY LN,None,FRANKLIN,TN,37067


In [109]:
fig = px.treemap(poach, path=['Facility_Group', 'Classification', 'Name'], values='NPI_Patients',
                color='NPI_Transactions', 
                hover_data=['Name', 'Classification', 'Specialization', 'provider_business_address_1',
                           'provider_business_address_2', 'provider_business_city',
                           'provider_business_state', 'provider_business_zip5'],
                color_continuous_scale='Emrld')
#fig.show()
#fig.write_html("/Users/mattparker/Documents/nss_projects/hcbb_hop_team-3m/notebooks/poaching_plotly.html")
#py.plot(fig, filename = 'poaching_plotly_published', auto_open=True)

# Percent Competition: Question 2/3 Visualization

In [164]:
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
        WITH vandy_selector AS (
            SELECT from_npi, patient_count, CASE
                WHEN to_facility_group = 'Vanderbilt University Medical Center' THEN 'vandy'
                ELSE 'not_vandy' END AS to_vandy,
                SUM(patient_count) OVER(PARTITION BY from_npi) AS total_patients
            FROM referrals
        ), percents AS (
            SELECT from_npi AS npi
                , SUM(patient_count) AS non_vandy_patients
                , total_patients
                , ROUND(100.0 * SUM(patient_count)/total_patients, 2) AS pct_competition
            FROM vandy_selector
            WHERE to_vandy = 'not_vandy'
            GROUP BY from_npi, to_vandy
        ), npi_details AS (
            SELECT npi
                , provider_first_name || ' ' || provider_last_name AS Name
                , classification AS Classification
                , COALESCE(specialization, 'None') AS Specialization
                , provider_business_address_1
                , provider_business_address_2
                , provider_business_city
                , provider_business_state
                , provider_business_zip5
            FROM npidata_nashville
            GROUP BY npi
        )
        SELECT *
        FROM percents
        JOIN npi_details USING(npi)
        WHERE classification NOT IN ('Emergency Medicine', 'Orthopaedic Surgery', 'Radiology',
                                            'Pathology', 'Anesthesiology')
        AND total_patients >= 500
        """ 

    test = pd.read_sql(query, db)

display(test)

,npi,non_vandy_patients,total_patients,pct_competition,Name,Classification,Specialization,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip5
0,1003819277,675,675,100.00,JOHN CAGE,Internal Medicine,Cardiovascular Disease,222 22ND AVE N,STE 400,NASHVILLE,TN,37203
1,1013179860,1165,1241,93.88,GARY YAWN,Internal Medicine,Interventional Cardiology,353 NEW SHACKLE ISLAND RD STE 300C,None,HENDERSONVILLE,TN,37075
2,1013908730,462,548,84.31,JOSEPH PARKER,Internal Medicine,Gastroenterology,222 22ND AVE N,None,NASHVILLE,TN,37203
3,1013958776,487,1044,46.65,AHMAD ABU-HALIMAH,Internal Medicine,Cardiovascular Disease,VANDERBILT UNIVERSITY MEDICAL CTR. CARDIOLOGY ...,"383 PRB, 2220 PIERCE AVE.",NASHVILLE,TN,37232
4,1023000999,557,626,88.98,CLARA WOMACK,Internal Medicine,Nephrology,393 WALLACE RD,STE. 203,NASHVILLE,TN,37211
...,...,...,...,...,...,...,...,...,...,...,...,...
310,1962656322,511,511,100.00,CANDICE OLECHOWSKI,Internal Medicine,None,1224 TROTWOOD AVE,None,COLUMBIA,TN,38401
311,1972658060,1878,1878,100.00,TERRY KETCH,Internal Medicine,Cardiovascular Disease,3443 DICKERSON PIKE STE 430,None,NASHVILLE,TN,37207
312,1982661617,562,680,82.65,CHRISTIE GREEN,Internal Medicine,Nephrology,1617 WILLIAMS DR,STE. 200,MURFREESBORO,TN,37129
313,1982796306,247,752,32.85,MATTHEW ABBATE,Internal Medicine,None,3601 TVC,None,NASHVILLE,TN,37232


In [166]:
test_vis = px.treemap(test, path=['Classification', 'Specialization', 'Name'], values='non_vandy_patients',
                color='pct_competition', 
                hover_data=['Name', 'Classification', 'Specialization', 'provider_business_address_1',
                           'provider_business_address_2', 'provider_business_city',
                           'provider_business_state', 'provider_business_zip5', 'total_patients'],
                color_continuous_scale='Emrld',
                labels=({"pct_competition": "% Competitor",
                        "provider_business_address_1": "Address Line 1",
                        "provider_business_address_2": "Address Line 2",
                        "provider_business_city": "City",
                        "provider_business_state": "State",
                        "provider_business_zip5": "Zip",
                        "total_patients": "Total Patients"})
)
test_vis.update_layout(title_text='Percent Patients Sent to Competitor Hospital Groups', title_x=0.5,
                                 title_font_size=24)
test_vis.show()
#percent_competition.write_html("/Users/mattparker/Documents/nss_projects/hcbb_hop_team-3m/notebooks/percent_competition_radiology.html")
py.plot(test_vis, filename = 'percent_competition', auto_open=True)


'https://plotly.com/~matttparker/6/'

In [156]:
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
        WITH vandy_selector AS (
            SELECT from_npi, SUM(patient_count) AS patient_count, CASE
                WHEN to_facility_group = 'Vanderbilt University Medical Center' THEN 'vandy'
                ELSE 'not_vandy' END AS to_vandy,
                SUM(patient_count) OVER(PARTITION BY from_npi) AS total_patients
            FROM referrals
            GROUP BY from_npi
        ), percents AS (
            SELECT from_npi AS npi
                , patient_count AS non_vandy_patients
                , total_patients
                , ROUND(100.0 * patient_count/total_patients, 2) AS pct_competition
            FROM vandy_selector
            WHERE to_vandy = 'not_vandy'
            GROUP BY from_npi, to_vandy
        ), npi_details AS (
            SELECT npi
                , provider_first_name || ' ' || provider_last_name AS Name
                , classification AS Classification
                , COALESCE(specialization, 'None') AS Specialization
                , provider_business_address_1
                , provider_business_address_2
                , provider_business_city
                , provider_business_state
                , provider_business_zip5
            FROM npidata_nashville
            GROUP BY npi
        )
        SELECT *
        FROM percents
        JOIN npi_details USING(npi)
        WHERE classification NOT IN ('Emergency Medicine', 'Orthopaedic Surgery', 'Radiology',
                                                'Pathology', 'Anesthesiology')
        AND total_patients >= 500
        """ 

    percents_treemap = pd.read_sql(query, db)

display(percents_treemap)

,npi,non_vandy_patients,total_patients,pct_competition,Name,Classification,Specialization,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip5
0,1023103462,1219,1219,100.00,THOMAS JOHNSTON,Internal Medicine,Advanced Heart Failure and Transplant Cardiology,2400 PATTERSON ST,SUITE 502,NASHVILLE,TN,37203
1,1043300288,572,572,100.00,ELISABETH WILLERS,Internal Medicine,Pulmonary Disease,3601 THE VANDERBILT CLINIC,None,NASHVILLE,TN,37232
2,1053337717,1824,1824,100.00,SUNIL KAZA,Internal Medicine,Cardiovascular Disease,3443 DICKERSON PIKE,SUITE 430,NASHVILLE,TN,37207
3,1053519785,1070,991,107.97,JONATHAN HODA,Internal Medicine,Cardiovascular Disease,5651 FRIST BLVD STE 603,None,HERMITAGE,TN,37076
4,1053608364,554,554,100.00,SERENA VAYDA,Hospitalist,None,2312 ELLISTON PL,None,NASHVILLE,TN,37203
...,...,...,...,...,...,...,...,...,...,...,...,...
144,1962481614,700,700,100.00,KEVIN CHUNN,Specialist,None,1226 TROTWOOD AVE,None,COLUMBIA,TN,38401
145,1962497867,573,573,100.00,ABU FAKHRUDDIN,Internal Medicine,Gastroenterology,330 23RD AVE N,SUITE 300,NASHVILLE,TN,37203
146,1962515023,729,596,122.32,BELINDA BART,Family Medicine,None,1605 NASHVILLE HWY,SUITE 200,COLUMBIA,TN,38401
147,1962656322,511,511,100.00,CANDICE OLECHOWSKI,Internal Medicine,None,1224 TROTWOOD AVE,None,COLUMBIA,TN,38401


In [120]:
percent_competition = px.treemap(percents_treemap, path=['Classification', 'Specialization', 'Name'], values='non_vandy_patients',
                color='pct_competition', 
                hover_data=['Name', 'Classification', 'Specialization', 'provider_business_address_1',
                           'provider_business_address_2', 'provider_business_city',
                           'provider_business_state', 'provider_business_zip5', 'total_patients'],
                color_continuous_scale='Emrld',
                labels=({"pct_competition": "% Competitor",
                        "provider_business_address_1": "Address Line 1",
                        "provider_business_address_2": "Address Line 2",
                        "provider_business_city": "City",
                        "provider_business_state": "State",
                        "provider_business_zip5": "Zip",
                        "total_patients": "Total Patients"})
)
percent_competition.update_layout(title_text='Percent Patients Sent to Competitor Hospital Groups', title_x=0.5,
                                 title_font_size=24)
percent_competition.show()
#percent_competition.write_html("/Users/mattparker/Documents/nss_projects/hcbb_hop_team-3m/notebooks/percent_competition_radiology.html")
#py.plot(percent_competition, filename = 'percent_competition', auto_open=True)


'https://plotly.com/~matttparker/6/'

# Alvin Request #1 (Alternate to Previous Visual) - NOT IN PRESENTATION

In [110]:
fig_1 = px.treemap(poach, path=['Classification', 'Specialization', 'Name'], values='NPI_Patients',
                color='NPI_Patients', 
                hover_data=['Name', 'Classification', 'Specialization', 'provider_business_address_1',
                           'provider_business_address_2', 'provider_business_city',
                           'provider_business_state', 'provider_business_zip5'],
                color_continuous_scale='Emrld')
#fig_1.show()

# Hospital to Specialty

In [117]:
# From below, summarize by Facility Group
# Find hospital groups with the largest number of single group, non-Vanderbilt NPIs
# Remove Radiology
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
        WITH CTE AS (
            SELECT from_npi
                , COUNT(DISTINCT to_facility_group) AS to_groups
                , SUM(transaction_count) AS npi_transactions
                , SUM(patient_count) AS npi_patients
                , to_facility_group
            FROM referrals
            WHERE from_npi_specialty NOT IN ('Emergency Medicine', 'Orthopaedic Surgery', 'Radiology',
                                            'Pathology', 'Anesthesiology')
            GROUP BY from_npi
        ), CTE_2 AS(
            SELECT CTE.to_facility_group, CTE.npi_transactions, CTE.npi_patients, n.*
            FROM CTE
            INNER JOIN npidata_nashville AS n 
                ON CTE.from_npi = n.npi
            ORDER BY CTE.npi_transactions DESC
        )
        SELECT to_facility_group AS Facility_Group
            , npi
            , provider_first_name || ' ' || provider_last_name AS Name
            , classification AS Classification
            , COALESCE(specialization, 'None') AS Specialization
            , npi_transactions AS NPI_Transactions
            , npi_patients AS NPI_Patients
            , provider_business_address_1
            , provider_business_address_2
            , provider_business_city
            , provider_business_state
            , provider_business_zip5
            , SUM(NPI_Patients) AS Total_Patients
        FROM CTE_2
        GROUP BY to_facility_group, classification, specialization
        """ 

    poach2 = pd.read_sql(query, db)

display(poach2)

,Facility_Group,npi,Name,Classification,Specialization,NPI_Transactions,NPI_Patients,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip5,Total_Patients
0,Ascension Saint Thomas,1497709752,ERIKA GILBERT,Anesthesiologist Assistant,None,87,84,1800 MEDICAL CENTER PKWY,SUITE 330,MURFREESBORO,TN,37129,84
1,Ascension Saint Thomas,1487780268,NORMA KRANTZ,Clinical Nurse Specialist,Women's Health,146,141,300 20TH AVE N STE 401,None,NASHVILLE,TN,37203,141
2,Ascension Saint Thomas,1952561474,SREE SURYADEVARA,Colon & Rectal Surgery,None,129,98,2011 CHURCH ST,STE. 703,NASHVILLE,TN,37203,226
3,Ascension Saint Thomas,1952304966,JOHN SALYER,Family Medicine,None,904,604,127 CRESTVIEW PARK DR,None,DICKSON,TN,37055,6116
4,Ascension Saint Thomas,1386878957,HOLLY BLANKENSHIP,Family Medicine,Adult Medicine,105,75,370 DOOLITTLE RD STE 1,None,WOODBURY,TN,37190,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,Williamson Medical Center,1538157219,LAURA BASKIN,Surgery,None,266,143,1272 GARRISON DR,None,MURFREESBORO,TN,37129,913
431,Williamson Medical Center,1174622500,MARK KELLEY,Surgery,Surgical Oncology,509,300,3601 THE VANDERBILT CLINIC,None,NASHVILLE,TN,37232,300
432,Williamson Medical Center,1750327052,CARY PULLIAM,Surgery,Vascular Surgery,691,385,4601 CAROTHERS PKWY,STE 375,FRANKLIN,TN,37067,949
433,Williamson Medical Center,1184657272,ERIC LAMBRIGHT,Thoracic Surgery (Cardiothoracic Vascular Surg...,None,492,258,3601 VANDERBILT CLINIC,None,NASHVILLE,TN,37232,258


In [122]:
hospital_to_specialty = px.treemap(poach2, path=['Facility_Group', 'Classification', 'Specialization'], values='Total_Patients',
                color='Total_Patients', 
                hover_data=['Facility_Group', 'Classification', 'Specialization'],
                color_continuous_scale='Emrld',
                labels=({"Facility_Group": "Facility Group",
                        "Total_Patients": "Total Patients"})
                )
hospital_to_specialty.update_layout(title_text='Referrals to Hospital Groups by Classification and Specialty'
                                    , title_x=0.5, title_font_size=20)
hospital_to_specialty.show()
#hospital_to_specialty.write_html("/Users/mattparker/Documents/nss_projects/hcbb_hop_team-3m/notebooks/hospital_to_specialty_radiology.html")
#py.plot(hospital_to_specialty, filename = 'Hospitals to Specialty', auto_open=True)


'https://plotly.com/~matttparker/8/'

# Alvin Request #3

In [131]:
# From below, summarize by Facility Group
# Find hospital groups with the largest number of single group, non-Vanderbilt NPIs
# Remove Radiology
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
        WITH CTE AS (
            SELECT from_npi
                , COUNT(DISTINCT to_facility_group) AS to_groups
                , SUM(transaction_count) AS npi_transactions
                , SUM(patient_count) AS npi_patients
                , TRIM(REPLACE('to_facility_group', 'Vanderbilt University Medical Center', 'VUMC'))
            FROM referrals
            WHERE from_npi_specialty NOT IN ('Emergency Medicine', 'Orthopaedic Surgery', 'Radiology',
                                            'Pathology', 'Anesthesiology')
                                            
            GROUP BY from_npi
            HAVING npi_transactions >= 500
        ), CTE_2 AS(
            SELECT CTE.to_facility_group, CTE.npi_transactions, CTE.npi_patients, n.*
            FROM CTE
            INNER JOIN npidata_nashville AS n 
                ON CTE.from_npi = n.npi
            ORDER BY CTE.npi_transactions DESC
        )
        SELECT to_facility_group AS Facility_Group
            , npi
            , provider_first_name || ' ' || provider_last_name AS Name
            , classification AS Classification
            , COALESCE(specialization, 'None') AS Specialization
            , npi_transactions AS NPI_Transactions
            , npi_patients AS NPI_Patients
            , provider_business_address_1
            , provider_business_address_2
            , provider_business_city
            , provider_business_state
            , provider_business_zip5
            , SUM(NPI_Patients) AS Total_Patients
        FROM CTE_2
        GROUP BY classification, specialization, to_facility_group
        """ 

    poach3 = pd.read_sql(query, db)

display(poach3)

DatabaseError: Execution failed on sql '
        WITH CTE AS (
            SELECT from_npi
                , COUNT(DISTINCT to_facility_group) AS to_groups
                , SUM(transaction_count) AS npi_transactions
                , SUM(patient_count) AS npi_patients
                , TRIM(REPLACE('to_facility_group', 'Vanderbilt University Medical Center', 'VUMC'))
            FROM referrals
            WHERE from_npi_specialty NOT IN ('Emergency Medicine', 'Orthopaedic Surgery', 'Radiology',
                                            'Pathology', 'Anesthesiology')
                                            
            GROUP BY from_npi
            HAVING npi_transactions >= 500
        ), CTE_2 AS(
            SELECT CTE.to_facility_group, CTE.npi_transactions, CTE.npi_patients, n.*
            FROM CTE
            INNER JOIN npidata_nashville AS n 
                ON CTE.from_npi = n.npi
            ORDER BY CTE.npi_transactions DESC
        )
        SELECT to_facility_group A

In [128]:
classification_to_hospital = px.treemap(poach3, path=['Classification', 'Specialization', 'Facility_Group'], values='Total_Patients',
                color='Total_Patients', 
                hover_data=['Classification', 'Specialization', 'Facility_Group'],
                color_continuous_scale='Emrld',
                labels=({"Facility_Group": "Facility Group",
                        "Total_Patients": "Total Patients"})
                )
classification_to_hospital.update_layout(title_text='Referrals to Classification by Hospital Group'
                                    , title_x=0.5, title_font_size=24)
classification_to_hospital.show()
#classification_to_hospital.write_html("/Users/mattparker/Documents/nss_projects/hcbb_hop_team-3m/notebooks/classification_to_hospital_radiology.html")
#py.plot(classification_to_hospital, filename = 'Classification to Hospitals', auto_open=True)


'https://plotly.com/~matttparker/11/'